*   **Año:** [2024]
*   **Alumno/a:** [PEDRO URRETAVIZCAYA]
*   **Legajo:** [LEGAJO]

# Pydantic
Pydantic es una libreria rapida y extensible que nos permite validar datos usando tipos de datos de Python.


Primero, importe `pydantic`.

In [72]:
from pydantic import BaseModel, Field, StrictInt, constr, validator, ValidationError
from typing import Literal
from datetime import date

Supongamos que tenemos una lista de clientes (llamemoslo en el codigo `Client`). Los clientes tienen dos campos: DNI (un entero) y nacionalidad (un `string`). Cree el modelo base de `Client` (en forma de clases de Python).

In [73]:
class Client(BaseModel):
  DNI: int
  nacionalidad: str

Cree a un usuario con documento 39.755.010 y nacionalidad 'Argentina'. Muestre todos sus campos.

In [74]:
cliente1 = Client(DNI=39755010, nacionalidad='Argentina')
cliente1

Client(DNI=39755010, nacionalidad='Argentina')

Intente crear al usuario con un documento en forma de `string`. Deberia fallar...

In [75]:
cliente2 = Client(DNI="39755010", nacionalidad="Argentina")
cliente2

Client(DNI=39755010, nacionalidad='Argentina')

Hemos detectado que ciertos clientes tienen nacionalidades que no existen. Ademas, hay numeros de documento negativos y se tiene que poder agregar la fecha de registro de los clientes (que no pueden ser del futuro). Cambiar la definicion del cliente para que estas cosas no ocurran. Despues de la siguiente celda, cree otras 3 mas probando un caso donde deberia funcionar y otros dos en los que no.

In [76]:
class Client(BaseModel):
    DNI: StrictInt = Field(..., gt=0)
    nacionalidad: Literal['Argentina', 'Brasil', 'Chile', 'Uruguay']
    fecha_registro: date

    @validator('fecha_registro')
    def check_fecha_registro(cls, value):
        if value > date.today():
            raise ValueError('error')
        return value

<ipython-input-76-9c067b7a4888>:6: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @validator('fecha_registro')


In [77]:
prueba1 = Client(DNI=39755010, nacionalidad='Argentina', fecha_registro=date(2024,10,10))
prueba1

Client(DNI=39755010, nacionalidad='Argentina', fecha_registro=datetime.date(2024, 10, 10))

In [78]:
prueba2 = Client(DNI=7777777, nacionalidad='Brasil', fecha_registro=date(2022,7,8))
prueba2

Client(DNI=7777777, nacionalidad='Brasil', fecha_registro=datetime.date(2022, 7, 8))

In [79]:
prueba3 = Client(DNI=999999, nacionalidad='Chile', fecha_registro=date(2020,9,9))
prueba3

Client(DNI=999999, nacionalidad='Chile', fecha_registro=datetime.date(2020, 9, 9))

In [80]:
try:
    prueba_mal1 = Client(DNI=39755010, nacionalidad='Argentina', fecha_registro=date(2025, 5, 5))
    print(prueba_mal1)
except ValidationError as e:
    print(f"Error de validación: {e}")

Error de validación: 1 validation error for Client
fecha_registro
  Value error, error [type=value_error, input_value=datetime.date(2025, 5, 5), input_type=date]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error


In [81]:
try:
    prueba_mal2 = Client(DNI=-39755010, nacionalidad='hola', fecha_registro=date(2026, 7, 8))
    print(prueba_mal2)
except ValidationError as e:
    print(f"Error de validación: {e}")

Error de validación: 3 validation errors for Client
DNI
  Input should be greater than 0 [type=greater_than, input_value=-39755010, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/greater_than
nacionalidad
  Input should be 'Argentina', 'Brasil', 'Chile' or 'Uruguay' [type=literal_error, input_value='hola', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/literal_error
fecha_registro
  Value error, error [type=value_error, input_value=datetime.date(2026, 7, 8), input_type=date]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error


# Pandera
Hacer lo que se pidio en la ultima celda anterior, pero con Pandera.

In [82]:
!pip install pandera

In [83]:
import pandera as pa
import pandas as pd
import numpy as np
from datetime import date

In [84]:
schema = pa.DataFrameSchema({
    "DNI": pa.Column(int, checks=pa.Check.ge(1)),
    "nacionalidad": pa.Column(str, checks=pa.Check.isin(["Argentina", "Brasil", "Chile", "Uruguay"])),
    "fecha_registro": pa.Column(pd.Timestamp, checks=pa.Check.le(pd.Timestamp(date.today())))
})

In [85]:
df1 = pd.DataFrame({
    "DNI": [39755010],
    "nacionalidad": ["Argentina"],
    "fecha_registro": pd.to_datetime([date(2024, 10, 10)])
})

try:
    schema.validate(df1)
    print("El DataFrame es válido.")
except pa.errors.SchemaErrors as e:
    print("error")

El DataFrame es válido.


In [91]:
df2 = pd.DataFrame({
    "DNI": [777777],
    "nacionalidad": ["hola"],
    "fecha_registro": pd.to_datetime([date(2022, 7, 8)])
})

try:
    schema.validate(df2)
    print("El DataFrame es válido.")
except pa.errors.SchemaErrors as e:
    print(f"Error de validación: {e}")

SchemaError: Column 'nacionalidad' failed element-wise validator number 0: isin(['Argentina', 'Brasil', 'Chile', 'Uruguay']) failure cases: hola

In [92]:
df3 = pd.DataFrame({
    "DNI": [-1],
    "nacionalidad": ["Brasil"],
    "fecha_registro": pd.to_datetime([date(2022, 7, 8)])
})

try:
    schema.validate(df2)
    print("El DataFrame es válido.")
except pa.errors.SchemaErrors as e:
    print("error")

SchemaError: Column 'nacionalidad' failed element-wise validator number 0: isin(['Argentina', 'Brasil', 'Chile', 'Uruguay']) failure cases: hola